## Game pattern and bye week constraints

In [1]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import *
import pandas as pd
import numpy as np


#list of teams that are included in the analysis
Team_list={
0:"Atlanta Falcons",1:"Carolina Panthers",2:"Chicago Bears",3:"Detroit Lions",
4:"Green Bay Packers",5:"Minnesota Vikings",6:"New Orleans Saints",7:"New York Giants",
8:"Philadelpia Eagles",9:"Tampa Bay Buccaneers",10:"Washington Football Team",11:"Baltimore Ravens",
12:"Buffalo Bills",13:"Cincinnati Bengals",14:"Cleveland Browns",15:"Houston Texans",
16:"Indianapolis Colts",17:"Jacksonville Jaguars",18:"Miami Dolphins",19:"New England Patroits",
20:"New York Jets",21:"Pittsburgh Steelers",22:"Tennessee Titans",23:"Dalls Cowboys"}


W=set(list(range(0,13))) #set of weeks, total of 12 weeks
T=set(list(range(0,24))) #set of teams all, total of 24 teams
#set of conference
D1=set(list(range(0,12)))  #set of AFC teams
D2=set(list(range(12,24))) #set of NFC teams

#read the distance file
E=pd.read_csv("distance.csv",index_col=0)
week = 13

In [3]:
# Create an empty model
m= gp.Model(name="NFL3")
# ADD DECISION VARIABLES HERE
x = m.addVars(T, T, W, vtype = GRB.BINARY, name = "X")

Using license file /Users/zhangmaojn/gurobi.lic
Academic license - for non-commercial use only


In [4]:
# ADD OBJECTIVE FUNCTION
# Objective is to minimize the total travel distance of all teams
obj = sum(E.iloc[i,j]*x[i,j,k] for i in T for j in T for k in W)*2

In [5]:
# ADD CONSTRAINTS
# add constraints that The season was limited to 12 weeks
m.addConstrs(sum(sum(x[i,j,k]+x[j,i,k] for k in W) for i in T) == 12 for j in T)
# add constraints that Each team would play once per week
m.addConstrs(sum(x[i,j,k]+x[j,i,k] for j in T) == 1 for i in T for k in range(0,8))
m.addConstrs(sum(x[i,j,k]+x[j,i,k] for j in T) == 1 for i in T for k in range(9,13))
# add constraints that All 12 games that a team played would need to be against a different opponent
m.addConstrs(sum(x[i,j,k]+x[j,i,k]for k in W) <= 1 for i in T for j in T)
# add constraints that Each team would play at most six home games (i.e., on their home stadium)
m.addConstrs(sum(sum(x[i,j,k] for k in W) for j in T) <=6 for i in T)
# add constraints to avoid circumstances that i=i
m.addConstrs(x[i,i,k] == 0 for i in T for i in T for k in W)
# add constraints to let binary variables are bound between 0 and 1
m.addConstrs(x[i,j,k] <= 1 for i in T for j in T for k in W)
m.addConstrs(x[i,j,k] >= 0 for i in T for j in T for k in W)

# add gaming pattern constraints for 2a
m.addConstrs(sum(x[i,j,k]+x[i,j,k+1]+x[i,j,k+2] for j in T) <= 2 for i in T for k in range(week-2))
m.addConstrs(sum(x[i,j,k]+x[i,j,k+1]+x[i,j,k+2] for j in T) >= 1 for i in T for k in range(week-2))

# add gaming pattern constraints for there are no games on week9
m.addConstrs(x[i,j,8] == 0 for i in T for j in T)
#3. Eachteam has a maximum distance to travel of 11000
m.addConstrs((sum(x[i,j,k] * E.iloc[i,j] * 2 for k in W for i in T) <= 11000
            for j in T))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

In [6]:
#optimize the model
m.setObjective(obj, GRB.MINIMIZE)
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 24504 rows, 7488 columns and 118558 nonzeros
Model fingerprint: 0xde87297c
Variable types: 0 continuous, 7488 integer (7488 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [7e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 23484 rows and 864 columns
Presolve time: 0.21s
Presolved: 1020 rows, 6624 columns, 59592 nonzeros
Variable types: 0 continuous, 6624 integer (6624 binary)

Root relaxation: objective 1.659080e+05, 9462 iterations, 1.67 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 165908.000    0  355          - 165908.000      -     -    4s
     0     0 165908.000    0  491          - 165908.000      -     -    5s
     0     0 165908.000    0  304          - 165908.000      -     -    8s
     0   

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

In [7]:
# ADD PRINTING HERE:

# print the total distance travelled by all teams
print('\nValue of objective function: %g' % m.objVal)

# print the optimal schedule for Cleveland Browns
#14:"Cleveland Browns"
rows=[]
for v in m.getVars():
    if v.varName.find("14")!= -1 and v.x==1:
        rows.append([v.varName, v.x])
print('\n Schedule for Cleveland Browns')
df = pd.DataFrame(rows, columns=["v.varName", "v.x"])
print(df)


# Q3: print distance that each team travels
distance_eachteam = {}

for a in Team_list:
    distance_travelled = 0
    for w in range(12):
        for h in range(24):
            distance_travelled += x[h,a,w].x*E.iloc[h,a]*2
        distance_eachteam[Team_list[a]] = distance_travelled

print(distance_eachteam)

keymax = max(distance_eachteam, key=distance_eachteam.get)
keymin = min(distance_eachteam, key=distance_eachteam.get)
print('\nTeam which travels the max distance:' +' '+ keymax)
print('\nTeam which travels the min distance:' +' '+  keymin)

Difference = distance_eachteam[max(distance_eachteam, key=distance_eachteam.get)]-distance_eachteam[min(distance_eachteam, key=distance_eachteam.get)]
print('\nDifference in distance travelled: %g' % Difference)



Value of objective function: 165908

 Schedule for Cleveland Browns
      v.varName  v.x
0     X[5,14,6]  1.0
1    X[11,14,5]  1.0
2    X[12,14,0]  1.0
3    X[13,14,3]  1.0
4     X[14,2,7]  1.0
5    X[14,3,10]  1.0
6     X[14,4,1]  1.0
7    X[14,7,12]  1.0
8    X[14,19,2]  1.0
9    X[14,21,4]  1.0
10   X[16,14,9]  1.0
11  X[20,14,11]  1.0
{'Atlanta Falcons': 6956.0, 'Carolina Panthers': 7040.0, 'Chicago Bears': 5322.0, 'Detroit Lions': 5238.0, 'Green Bay Packers': 7786.0, 'Minnesota Vikings': 9968.0, 'New Orleans Saints': 7642.0, 'New York Giants': 4534.0, 'Philadelpia Eagles': 6850.0, 'Tampa Bay Buccaneers': 7744.0, 'Washington Football Team': 5978.0, 'Baltimore Ravens': 3904.0, 'Buffalo Bills': 4432.0, 'Cincinnati Bengals': 6766.0, 'Cleveland Browns': 4672.0, 'Houston Texans': 8508.0, 'Indianapolis Colts': 6364.0, 'Jacksonville Jaguars': 6324.0, 'Miami Dolphins': 9904.0, 'New England Patroits': 5952.0, 'New York Jets': 3206.0, 'Pittsburgh Steelers': 3778.0, 'Tennessee Titans': 6208.

In [8]:
# Create an empty model
m= gp.Model(name="NFL3")
# ADD DECISION VARIABLES HERE
x = m.addVars(T, T, W, vtype = GRB.BINARY, name = "X")
obj = sum(E.iloc[i,j]*x[i,j,k] for i in T for j in T for k in W)*2

# ADD CONSTRAINTS
# add constraints that The season was limited to 12 weeks
m.addConstrs(sum(sum(x[i,j,k]+x[j,i,k] for k in W) for i in T) == 12 for j in T)
# add constraints that Each team would play once per week
m.addConstrs(sum(x[i,j,k]+x[j,i,k] for j in T) == 1 for i in T for k in range(0,8))
m.addConstrs(sum(x[i,j,k]+x[j,i,k] for j in T) == 1 for i in T for k in range(9,13))
# add constraints that All 12 games that a team played would need to be against a different opponent
m.addConstrs(sum(x[i,j,k]+x[j,i,k]for k in W) <= 1 for i in T for j in T)
# add constraints that Each team would play at most six home games (i.e., on their home stadium)
m.addConstrs(sum(sum(x[i,j,k] for k in W) for j in T) <=6 for i in T)
# add constraints to avoid circumstances that i=i
m.addConstrs(x[i,i,k] == 0 for i in T for i in T for k in W)
# add constraints to let binary variables are bound between 0 and 1
m.addConstrs(x[i,j,k] <= 1 for i in T for j in T for k in W)
m.addConstrs(x[i,j,k] >= 0 for i in T for j in T for k in W)

# add gaming pattern constraints for 2a
m.addConstrs(sum(x[i,j,k]+x[i,j,k+1]+x[i,j,k+2] for j in T) <= 2 for i in T for k in range(week-2))
m.addConstrs(sum(x[i,j,k]+x[i,j,k+1]+x[i,j,k+2] for j in T) >= 1 for i in T for k in range(week-2))

# add gaming pattern constraints for there are no games on week9
m.addConstrs(x[i,j,8] == 0 for i in T for j in T)
#3. Eachteam has a maximum distance to travel of 10990
m.addConstrs((sum(x[i,j,k] * E.iloc[i,j] * 2 for k in W for i in T) <= 10990
            for j in T))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

In [9]:
#optimize the model
m.setObjective(obj, GRB.MINIMIZE)
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 24504 rows, 7488 columns and 118558 nonzeros
Model fingerprint: 0x359a5e1a
Variable types: 0 continuous, 7488 integer (7488 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [7e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 23484 rows and 864 columns
Presolve time: 0.21s
Presolved: 1020 rows, 6624 columns, 59592 nonzeros
Variable types: 0 continuous, 6624 integer (6624 binary)

Root relaxation: objective 1.659080e+05, 9261 iterations, 1.39 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 165908.000    0  310          - 165908.000      -     -    4s
     0     0 165908.000    0  495          - 165908.000      -     -    5s
     0     0 165908.000    0  326          - 165908.000      -     -    7s
     0   

In [10]:
# ADD PRINTING HERE:

# print the total distance travelled by all teams
print('\nValue of objective function: %g' % m.objVal)

# print the optimal schedule for Cleveland Browns
#14:"Cleveland Browns"
rows=[]
for v in m.getVars():
    if v.varName.find("14")!= -1 and v.x==1:
        rows.append([v.varName, v.x])
print('\n Schedule for Cleveland Browns')
df = pd.DataFrame(rows, columns=["v.varName", "v.x"])
print(df)


# Q3: print distance that each team travels
distance_eachteam = {}

for a in Team_list:
    distance_travelled = 0
    for w in range(12):
        for h in range(24):
            distance_travelled += x[h,a,w].x*E.iloc[h,a]*2
        distance_eachteam[Team_list[a]] = distance_travelled

print(distance_eachteam)

keymax = max(distance_eachteam, key=distance_eachteam.get)
keymin = min(distance_eachteam, key=distance_eachteam.get)
print('\nTeam which travels the max distance:' +' '+ keymax)
print('\nTeam which travels the min distance:' +' '+  keymin)

Difference = distance_eachteam[max(distance_eachteam, key=distance_eachteam.get)]-distance_eachteam[min(distance_eachteam, key=distance_eachteam.get)]
print('\nDifference in distance travelled: %g' % Difference)



Value of objective function: 165908

 Schedule for Cleveland Browns
      v.varName  v.x
0     X[4,14,5]  1.0
1     X[5,14,0]  1.0
2   X[11,14,12]  1.0
3    X[14,2,11]  1.0
4     X[14,3,1]  1.0
5     X[14,7,6]  1.0
6    X[14,12,3]  1.0
7    X[14,13,9]  1.0
8    X[14,21,4]  1.0
9    X[16,14,7]  1.0
10  X[19,14,10]  1.0
11   X[20,14,2]  1.0
{'Atlanta Falcons': 3966.0, 'Carolina Panthers': 6938.0, 'Chicago Bears': 4616.0, 'Detroit Lions': 4088.0, 'Green Bay Packers': 9762.0, 'Minnesota Vikings': 9866.0, 'New Orleans Saints': 6446.0, 'New York Giants': 4970.0, 'Philadelpia Eagles': 7598.0, 'Tampa Bay Buccaneers': 7266.0, 'Washington Football Team': 5588.0, 'Baltimore Ravens': 4814.0, 'Buffalo Bills': 4944.0, 'Cincinnati Bengals': 5374.0, 'Cleveland Browns': 5432.0, 'Houston Texans': 8622.0, 'Indianapolis Colts': 5886.0, 'Jacksonville Jaguars': 7020.0, 'Miami Dolphins': 10464.0, 'New England Patroits': 5220.0, 'New York Jets': 3022.0, 'Pittsburgh Steelers': 2944.0, 'Tennessee Titans': 6448